# RAG Pipeline: Evaluation and Analysis

This notebook focuses on running the complete Retrieval-Augmented Generation (RAG) pipeline on a set of representative questions. We will analyze how the system retrieves relevant complaints and generates answers using the LLM.

### Pipeline Components:
1. **Vector Store**: FAISS index containing complaint embeddings.
2. **Retriever**: Similarity search mechanism to find top-K relevant documents.
3. **LLM**: Instruction-tuned model (FLAN-T5-Small) to generate answers based on context.
4. **Prompt Engineering**: Structured templates to guide the LLM's response.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import IPython
from IPython.display import display, Markdown

# allow imports from project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src import config, vectorstore, llm
from src.rag_pipeline import RAGPipeline

config.setup_hf_cache()
print("✓ Environment setup complete")

c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[OK] HuggingFace cache set to: c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\models\hf
✓ Environment setup complete


## 1. Initialize Production Pipeline

We initialize the `RAGPipeline` class, which encapsulates the loading of the vector store and the LLM engine.

In [2]:
pipeline = RAGPipeline()
if pipeline.initialize():
    print("✓ RAG Pipeline initialized and ready!")
else:
    print("❌ Pipeline initialization failed. Check your vector store and model paths.")


--- Initializing RAG Pipeline ---
Attempting to load vector store from: c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\vector_store\faiss
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
  (First run will download ~80MB to c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\models\hf)


Loading weights: 100%|██████████| 103/103 [00:01<00:00, 94.85it/s, Materializing param=pooler.dense.weight]                              
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[OK] Embedding model loaded
Loading existing FAISS index from c:\Users\Acer\Documents\KAIM_PROJECT\TEST\rag-complaint-chatbot\vector_store\faiss...
[OK] Vector store loaded (ntotal=1,375,327)
Loading Alternative LLM: google/flan-t5-small
  Max new tokens: 512
  Temperature: 0.1


Loading weights: 100%|██████████| 190/190 [00:02<00:00, 82.51it/s, Materializing param=shared.weight]                                                        
The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
Passing `generation_config` together with generation-related arguments=({'temperature', 'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM',

--- Pipeline Ready ---

✓ RAG Pipeline initialized and ready!


## 2. Representative Evaluation Questions

Here is a curated list of questions that represent common consumer concerns. We will use these to evaluate the system's performance.

In [3]:
eval_questions = [
    "What are common issues consumers face with credit card interest rates?",
    "How do customers describe problems with mortgage loan modifications?",
    "What are the most frequent complaints regarding unauthorized transactions on checking accounts?",
    "What reasons do consumers give for disputes regarding credit reporting inaccuracies?",
    "What are the common themes in complaints against debt collection agencies' practices?",
    "How do consumers react to unexpected fees on prepaid cards?",
    "What are the typical complaints about vehicle loan servicing?",
    "What issues do consumers report when trying to close their savings accounts?"
]

print(f"✓ Defined {len(eval_questions)} representative questions.")

✓ Defined 8 representative questions.


## 3. Running RAG Pipeline and Results Analysis

For each question, we run the full RAG cycle:
1. **Retrieve**: Find top 5 relevant complaint chunks.
2. **Augment**: Format the chunks into a context string for the prompt.
3. **Generate**: Use FLAN-T5-Small to generate a professional answer based *only* on the context.

In [4]:
all_results = []

for i, query in enumerate(eval_questions, 1):
    display(Markdown(f"### Question {i}: {query}"))
    
    # Run the pipeline
    result = pipeline.run(query, k=5)
    
    # Display the answer
    display(Markdown(f"**Generated Answer:** {result['answer']}"))
    
    # Display source metadata summary
    sources = result['source_documents']
    display(Markdown(f"*Retrieved {len(sources)} sources from products: {set([doc.metadata.get('product', 'N/A') for doc in sources])}*"))
    
    # Show a snippet of the most relevant source
    top_source = sources[0]
    display(Markdown(f"**Top Source Snippet (Complaint {top_source.metadata.get('complaint_id')}):**  \n*{top_source.page_content[:250]}...*"))
    
    display(Markdown("---"))
    
    all_results.append({
        "question": query,
        "answer": result['answer'],
        "sources": sources
    })

### Question 1: What are common issues consumers face with credit card interest rates?

Processing Query: 'What are common issues consumers face with credit card interest rates?'
Generating answer based on 5 sources...


Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


**Generated Answer:** You are a helpful financial assistant. Answer the question using ONLY the following context.

Guidelines:
1. Use ONLY the provided context to answer the question.
2. If the answer is not in the context, say "I don't have enough information to answer this based on the available complaints."
3. Maintain a helpful and professional tone.
4. If multiple pieces of information are relevant, summarize them clearly.

Context:
Content: but not going through where the big issues are ... the interest rates that these greedy credit card companies retail store cards are charging. these are the ones to be looked upon with more scrutiny. not the math models that you develop and ask banks to provide on frl issues, which are anyways.

Content: as a good time for them to raise their rates also. i mentioned to him that this seems like interest rate price fixing and collusion within these credit cards to raise interest rates arbitrarily because everyone else is doing it. what protections does a consumer have?

Content: problems are interest fee in which it causes the credit card company to be overcharged causing my monthly payment to increase greatly.

Content: es we include and the markets where we offer them. we continually review our rates to asses their competitiveness '' exactly what products '' are they referring to. we are all aware of the price increases all over in our country, if the credit card company 's increase their interest rates along with the cost of the products we buy, such as groceries ... ..... i will not be the only one hitting bottom. i literally have no money right now. we have about 40.00 between us.

Content: my complaint is an umbrella complaint about consumer credit card rates across the board. companies like synchrony bank, , and especially are charging way too much interest on consumer credit cards. for example, i have a branded from synchrony bank. when i got the card, i had a credit score of less than . the interest rate was . however, i have had the card for more than two years, i have had 100 on-time payment history, up to and including making extra payments.

Question: What are common issues consumers face with credit card interest rates?

Answer:

*Retrieved 5 sources from products: {'Credit card or prepaid card', 'Credit card'}*

...

Both `max_new_tokens` (=512) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


**Generated Answer:** You are a helpful financial assistant. Answer the question using ONLY the following context.

Guidelines:
1. Use ONLY the provided context to answer the question.
2. If the answer is not in the context, say "I don't have enough information to answer this based on the available complaints."
3. Maintain a helpful and professional tone.
4. If multiple pieces of information are relevant, summarize them clearly.

Context:
Content: care that i closed the saving account on the spot. the only thing that i was not surprised of is when the customer support showed no indifference in her attitude as i asked to close the account. one would think that a company would try to retain a customer of over 12 years but apparently not bank of america.

Content: on , i attempted to begin the process of closing my savings account, as well as a small investment account that i currently have through the online finance app called albert. though that process did not need to be time sensitive, the amount of roadblocks i have encountered in simply beginning has been, i think, beyond what any regulation would consider reasonable in regards to consumer-accessible closing and managing of financial products and accounts.

Content: after numerous banking errors on atm machines, fees, confidentiality and customer service issues. i decided to close both my checking and savings account.

Content: es with me on my account and i was forced to close my checking and savings account after being with that bank for over 20 years. since that just happened i am not providing details on that unless otherwise requested because it is kind of difficult to prove what they were doing. i do not have the ability to attached all the documentation that i have to support this complaint.

Content: e the accounts on our accounts remain open as of . this includes the savings account which has been at a balance since with multiple requests to close it.

Question: What issues do consumers report when trying to close their savings accounts?

Answer:

*Retrieved 5 sources from products: {'Money transfer, virtual currency, or money service', 'Checking or savings account'}*

**Top Source Snippet (Complaint 13049257):**  
*care that i closed the saving account on the spot. the only thing that i was not surprised of is when the customer support showed no indifference in her attitude as i asked to close the account. one would think that a company would try to retain a cu...*

---

## 4. Pipeline Evaluation Table

We consolidate the results into a structured evaluation table. This can be exported to your report. 

> **Note:** The `Quality Score` and `Comments/Analysis` columns are intended for manual review to ensure the AI's answers are grounded and accurate.

In [ ]:
evaluation_data = []

for res in all_results:
    # Get 1-2 source IDs
    source_ids = [doc.metadata.get('complaint_id', 'N/A') for doc in res['sources'][:2]]
    
    evaluation_data.append({
        "Question": res['question'],
        "Generated Answer": res['answer'],
        "Retrieved Sources": ", ".join(map(str, source_ids)),
        "Quality Score (1-5)": "",  # Placeholder for manual review
        "Comments/Analysis": ""      # Placeholder for manual review
    })

df_eval = pd.DataFrame(evaluation_data)

# Set column display for better readability in the notebook
pd.set_option('display.max_colwidth', 300)
display(df_eval)

